# Pandas 循环优化

Ref：[Pandas初学者代码优化指南](http://python.jobbole.com/88915/?utm_source=blog.jobbole.com&utm_medium=relatedPosts)

In [6]:
import pandas as pd
import numpy as np

data = {"name":["yahoo","google","facebook"], 
        "marks":[200,400,800], 
        "price":[9, 3, 7]} 
df = pd.DataFrame(data) 
columns = ['name','marks','price']
#df.columns = ['name','marks','price'] #该方式只是把列名改了，而没有重新排列的顺序
df = df.reindex_axis(columns, axis=1)
df = df.sort_values(by = 'price').reset_index(drop=True)  # 按price列升序
df

,name,marks,price
0,google,400,3
1,facebook,800,7
2,yahoo,200,9


In [7]:
import numpy as np
#@profile
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

# 1. Crude looping(粗糙的循环)——或者你永远不应该这么做
---
首先，让我们快速回顾一下Pandas数据结构的基本原理。Pandas的基本结构有两种形式：DataFrame和Series。一个DataFrame是一个二维数组标记轴，很多功能与R中的data.frame类似，可以将DataFrame理解为Series的容器。换句话说，一个DataFrame是一个有行和列的矩阵，列有列名标签，行有索引标签。在Pandas DataFrame中一个单独的列或者行是一个Pandas Series—一个带有轴标签的一维数组。

几乎每一个与我合作过的Pandas初学者，都曾经试图通过一次一个的遍历DataFrame行去应用自定义函数。这种方法的优点是，它是Python对象之间交互的一致方式；例如，一种可以通过列表或数组循环的方式。反过来说，不利的一面是，在Pandas中，Crude loop是最慢的方法。与下面将要讨论的方法不同，Pandas中的Crude loop没有利用任何内置优化，通过比较，其效率极低（而且代码通常不那么具有可读性）

例如，有人可能会写像下面这样的代码：

In [8]:
# Define a function to manually loop over all rows and return a series of distances
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['marks'], df.iloc[i]['price'])
        distance_list.append(d)
    return distance_list 

为了了解执行上述函数所需要的时间，我们用 **%timeit** 命令。
> * **%timeit** 是一个“神奇的”命令，专用于Jupyter notebook
> * **%timeit** 命令将多次运行一个函数，并打印出获得的运行时间的平均值和标准差。
> * 所有的魔法命令都以%标识开始，如果%命令只应用于一行，那么%%命令应用于整个Jupyter单元

当然，通过%timeit命令获得的运行时间，运行该函数的每个系统都不尽相同。尽管如此，它可以提供一个有用的基准测试工具，用于比较同一系统和数据集上不同函数的运行时间。

In [9]:
%%timeit
# Run the haversine looping function
df['distance'] = haversine_looping(df)

The slowest run took 5.86 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 824 µs per loop


# 2. iterrows()循环
---
如果循环是必须的，找一个更好的方式去遍历行，比如用iterrows（）方法。iterrows()是一个生成器，遍历DataFrame的所有行并返回每一行的索引，除了包含行自身的对象。iterrows() 是用Pandas DataFrame优化，尽管它是运行大多数标准函数最不高效的方式（稍后再谈），但相对于Crude looping，这是一个重大的改进。在我们的案例中，iterrows()解决同一个问题，几乎比手动遍历行快四倍。

In [10]:
%%timeit
# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985, row['marks'], row['price']))
df['distance'] = haversine_series

1000 loops, best of 3: 382 µs per loop


# 3. apply()——更好的循环
------
一个比iterrows()更好的选择是用 apply() 方法，它应用一个函数，沿着DataFrame某一个特定的轴线（意思就是行或列）。虽然apply()也固有的通过行循环，但它通过采取一些内部优化比iterrows()更高效，例如在Cython中使用迭代器。我们使用一个匿名的lambda函数，每一行都用Haversine函数，它允许指向每一行中的特定单元格作为函数的输入。为了指定Pandas是否应该将函数应用于行（axis = 1）或列（axis = 0），Lambda函数包含最终的axis参数。

In [11]:
%%timeit
# Timing apply on the Haversine function
df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['marks'], row['price']), axis=1)

The slowest run took 51.10 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 400 µs per loop


In [12]:
import line_profiler
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [16]:
#import lineprofiler
%lprun -f haversine df.apply(lambda row: haversine(40.671, -73.985, row['latitude'], row['longitude']), axis=1)

SyntaxError: unexpected character after line continuation character (<string>, line 1)

# 4. Pandas Series矢量化
---
要了解如何可以减少函数所执行的迭代数量，就要记得Pandas的基本单位，DataFrame和Series，它们都基于数组。基本单元的固有结构转换成内置的设计用于对整个数组进行操作的Pandas函数，而不是按各个值的顺序（简称标量）。矢量化是对整个数组执行操作的过程。

Pandas包含一个总体的矢量化函数集合，从数学运算到聚合和字符串函数（可用函数的扩展列表，查看Pandas docs）。对Pandas Series和DataFrame的操作进行内置优化。结果，使用矢量Pandas函数几乎总是会用自定义的循环实现类似的功能。

In [25]:
%%timeit
# Vectorized implementation of Haversine applied on Pandas series
df['distance'] = haversine(40.671, -73.985,df['marks'], df['price'])

1000 loops, best of 3: 1 ms per loop


# 5. 用NumPy数组矢量化
---
Pandas series矢量化可以完成日常计算优化的绝大多数需要。然而，如果速度是最高优先级，那么可以以NumPy Python库的形式调用援军。

NumPy库，将自己描述为一个“Python科学计算的基本包”，在后台执行优化操作，预编译C语言代码。跟Pandas一样，NumPy操作数组对象（简称ndarrays）；然而，它省去了Pandas series操作所带来的大量资源开销，如索引、数据类型检查等。因此，NumPy数组的操作可以明显快于pandas series的操作。

当Pandas series提供的额外功能不是很关键的时候，NumPy数组可以用于替代Pandas series。例如，Haversine函数矢量化实现不使用索引的经度和纬度系列，因此没有那些索引，也不会导致函数中断。通过比较，我们所做的操作如DataFrame的连接，它需要按索引来引用值，可能需要坚持使用Pandas对象。

仅仅是使用Pandas series 的values的方法，把纬度和经度数组从Pandas series转换到NumPy数组。就像series矢量化一样，通过NumPy数组直接进入函数将可以让Pandas对整个矢量应用函数。

In [26]:
%%timeit 
# Vectorized implementation of Haversine applied on NumPy arrays
df['distance'] = haversine(40.671, -73.985,df['marks'].values, df['price'].values)

10000 loops, best of 3: 138 µs per loop


![cmd-markdown-logo](http://wx4.sinaimg.cn/mw690/63918611gy1fls2tcpct6j20sm0bewfl.jpg)